<h1><center> Buscando dados de vídeos do YouTube - Parte 1 </center></h1>

<img src="youtube.com_.jpg" width="480">

### Objetivo
Apresentar um exemplo de como baixar dados de vídeos de um canal do YouTube via webscraping utilizando Selenium e Beautifulsoup. Neste caso, foi utilizado o canal de humor "Porta dos Fundos"

<b> Obs: Existem bibliotecas e a própria API do YouTube que simplificam bastante este processo, entretanto o objetivo deste tutorial é exemplificar como realizar esta tarefa utilizando exclusivamente webscraping. </b>

Para importar nossa base de dados vamos utilizar a seguinte estratégia:<br>
1 - Abrir a página de vídeos do canal desejado e rolar até o final para retornar a lista de vídeos publicados no canal desejado;<br>
2 - em seguida, será realizado um loop através dos vídeos para obter informações como título, visualizações, elenco, diretor(a), roteiristas, likes, dislikes e data de publicação via webscraping, as quais serão armazenadas em um dicionário;<br>
3 - após isto, os dados serão carregados em um pandas dataframe e se procederá às devidas limpezas para se obter a base de dados final, pronta para análise.
<br>
<br>
Vamos lá!!!

### Importando as libraries necessárias

In [1]:
import requests
import re
from bs4 import BeautifulSoup as bs
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Abrindo a página de vídeos do canal "Porta dos Fundos" a fim de carregar os links de todos eles

In [2]:
# definindo esta função para rolar a página de vídeos até o final a fim de renderizar todos os links
def scroll_down(current_elements_count, driver, scroll_pause_time):
    driver.execute_script('window.scrollBy(0, 50000);')
    time.sleep(scroll_pause_time)
    new_elements_count = driver.execute_script('return document.querySelectorAll("ytd-grid-video-renderer").length')
    if new_elements_count == current_elements_count:
        time.sleep(scroll_pause_time * 2)
        new_elements_count = driver.execute_script('return document.querySelectorAll("ytd-grid-video-renderer").length')
        if new_elements_count == current_elements_count:
            print('Reached end of page!')
    return new_elements_count

channel = "https://www.youtube.com/c/PortadosFundos/videos"

driver = webdriver.Chrome()  
driver.get(channel)

scroll_pause_time = 3
current_elements_count = None
new_elements_count  = driver.execute_script('return document.querySelectorAll("ytd-grid-video-renderer").length')

while new_elements_count != current_elements_count:
    current_elements_count = new_elements_count
    new_elements_count  = scroll_down(current_elements_count, driver, scroll_pause_time)

soup = bs(driver.page_source, "html.parser")
driver.close()

Reached end of page!


### Extraindo as tags com todos os ID's e títulos dos vídeos

In [3]:
find_all_a = soup.find_all('a',{ "class": "yt-simple-endpoint style-scope ytd-grid-video-renderer"})
len(find_all_a)

1659

### Extraindo títulos e ID's, formando os links e guardando tudo em um dicionário

In [4]:
videos = {"title":[], "url":[]}
for element in find_all_a:
    videos["title"].append(element.get_text())
    videos["url"].append("https://www.youtube.com"+element['href'])  
    
len(videos["title"])

1659

###  Carregando o dicionário em um dataframe e observando os primeiros registros

In [5]:
df = pd.DataFrame.from_dict(videos)
df.head()

,title,url
0,EX-FUNCIONÁRIO,https://www.youtube.com/watch?v=5-D_GCwNHFk
1,VACINADA,https://www.youtube.com/watch?v=WxzlRVq_gpI
2,PLANTANANÃ - TUDO SOBRE A CPI DA COVID,https://www.youtube.com/watch?v=E57UY6M8zo4
3,PFIZER,https://www.youtube.com/watch?v=PWwnNtl2Nyg
4,PREOCUPADÃO COM A COVID #EuMeCuido - #Shorts,https://www.youtube.com/watch?v=0Prrv9jXgls


### Excluindo os vídeos de making of para reduzir dupla contagem de atores/diretores/roteiristas na mesma peça

In [6]:
df_2 = df[~df['title'].str.contains('MAKING OF')]
df_2.shape

(1418, 2)

### Gerando uma lista com as URL's dos vídeos

In [7]:
url_list = df_2["url"].tolist()
url_list[0:10]
len(url_list)

['https://www.youtube.com/watch?v=5-D_GCwNHFk',
 'https://www.youtube.com/watch?v=WxzlRVq_gpI',
 'https://www.youtube.com/watch?v=E57UY6M8zo4',
 'https://www.youtube.com/watch?v=PWwnNtl2Nyg',
 'https://www.youtube.com/watch?v=0Prrv9jXgls',
 'https://www.youtube.com/watch?v=mwzp_UZrNMo',
 'https://www.youtube.com/watch?v=PfO5oPe___4',
 'https://www.youtube.com/watch?v=ssB7KMSUuwk',
 'https://www.youtube.com/watch?v=xsLrkt3XEaM',
 'https://www.youtube.com/watch?v=onNxHMHBdoA']

1418

### Realizando loop na lista de vídeos para baixar os dados desejados e guadá-los em um dicionário

In [ ]:
# incializando o dicionário que irá guardar as informações extraídas dos vídeos
result = {"title":[], "views":[], "cast":[], "script":[], "directing":[], "date_published":[], "likes":[], "dislikes":[]}

n = 0

#Esta função verifica se a conexão com a internet está ativa
def has_connection2(url):
    timeout = 5
    try:
        request = requests.get(url, timeout=timeout)
        return True
    except (requests.ConnectionError, requests.Timeout) as exception:
        return False
    
for url in url_list:
    print(n)
    print(url)
    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    # init webdriver
    driver = webdriver.Chrome(options=options)
    
    while has_connection2(url) == False:
        try:
            print("No internet connection.")            
            driver.close()
            time.sleep(2)
            driver = webdriver.Chrome()
        except:
             pass
            
    driver.get(url)
    time.sleep(4)

    # create beautiful soup object to parse HTML            
    soup = bs(driver.page_source, "html.parser")    
    description = str(driver.execute_script('return ytInitialPlayerResponse').get('microformat').get('playerMicroformatRenderer').get('description'))
    driver.close()
    
    # Padronizando alguns termos
    for character in ['Elenco', 'Elenco:', 'elenco', 'ELENCO:', "ELENCO "]:
        description = description.replace(character, "ELENCO")

    for character in ['Roteiro', 'Roteiro:', 'roteiro', 'ROTEIRO:', "ROTEIRO "]:
        description = description.replace(character, "ROTEIRO")

    for character in ['Direção', 'Direção:', 'direção', 'DIREÇÃO:', "DIREÇÃO "]:
        description = description.replace(character, "DIREÇÃO")

    for character in [' \\n ', ' \\n', '\\n ']:
        description = description.replace(character, "\\n")

    #title
    result["title"].append(soup.find("h1").text.strip())
    #views
    result["views"].append(soup.find("span", attrs={"class": "view-count"}).text)
    #cast
    result["cast"].append(''.join(re.findall(r'\\nELENCO\\n(.*?)\\n\\n', description)))
    #script
    result["script"].append(''.join(re.findall(r'\\nROTEIRO\\n(.*?)\\n\\n', description)))
    #directing
    result["directing"].append(''.join(re.findall(r'\\nDIREÇÃO\\n(.*?)\\n\\n', description)))
    # date published
    result["date_published"].append(soup.find("meta", {"itemprop":"datePublished"})['content'])
    #likes
    elem_likes_dislikes = soup.find_all("yt-formatted-string", {"id": "text", "class": "ytd-toggle-button-renderer"})
    str_like = ''.join(re.findall(r'yt-formatted-string(.*?)class="style-scope',str(elem_likes_dislikes[0]).replace(".","")))
    result["likes"].append(''.join(re.findall('\d+', str_like)))
    #dislikes
    str_dislike = ''.join(re.findall(r'yt-formatted-string(.*?)class="style-scope',str(elem_likes_dislikes[1]).replace(".","")))
    result["dislikes"].append(''.join(re.findall('\d+', str_dislike)))
    n = n + 1

### Carregando os dados em um dataframe para realizar as limpezas necessárias

In [21]:
#df = pd.read_csv('porta_dos_fundos_long_raw.csv')
df = pd.DataFrame.from_dict(result)
df.shape
df.tail()

(1418, 8)

,title,views,cast,script,directing,date_published,likes,dislikes
1413,BATMAN: THE DARK KNIGHT ERECTS,9.650.450 visualizações,Antonio Tabet\nFabio Porchat\nVitor Costa\nRaf...,Antonio Tabet\nGabriel Totoro\nGabriel Esteves...,Ian SBF,2012-08-09,172821.0,4519.0
1414,CSI NOVA IGUAÇU #8,1.183.997 visualizações,Antonio Tabet\nGabriel Totoro\nGustavo Chagas\...,Antonio Tabet,Ian SBF,2012-08-07,NaN,NaN
1415,COCAÍNA - Porta Dos Fundos Nº 1,2.325.550 visualizações,Julia Rabello\nAntonio Tabet,NaN,NaN,2012-08-07,NaN,NaN
1416,TRAVECO DA FIRMA - Porta dos Fundos Nº 1,12.198.167 visualizações,Fabio Porchat\nGustavo Chagas\nRafael Infante,NaN,NaN,2012-08-07,NaN,NaN
1417,PORTA DOS FUNDOS N°1,8.045.907 visualizações,NaN,NaN,NaN,2012-08-06,NaN,NaN


### Extraindo a quantidade de visualizações da variável "views" e convertendo em numérico

In [22]:
df['views_num'] = df.views.str.extract(r'(\d+[.\d]*)')
df['views_num'] = df.views_num.str.replace('.', '').astype(int)

<ipython-input-22-8ce3324039c1>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['views_num'] = df.views_num.str.replace('.', '').astype(int)


In [23]:
df.head()
df.info()

,title,views,cast,script,directing,date_published,likes,dislikes,views_num
0,EX-FUNCIONÁRIO,404.536 visualizações,Evelyn Castro\nEstevam Nabote\nNoemia Oliveira,Pedro Esteves,Zaga Martelletto,2021-05-24,46470.0,379.0,404536
1,VACINADA,527.924 visualizações,Thati Lopes,Gregório Duvivier,Bianca Frossard,2021-05-22,64951.0,1596.0,527924
2,PLANTANANÃ - TUDO SOBRE A CPI DA COVID,322.293 visualizações,Rafael Infante,João Vicente de Castro,Vini Videla,2021-05-21,44357.0,1522.0,322293
3,PFIZER,888.119 visualizações,Antonio Tabet\nFábio Porchat\nJoão Vicente de ...,Gregório Duvivier,Rodrigo Van Der Put,2021-05-20,126596.0,3800.0,888119
4,PREOCUPADÃO COM A COVID #EuMeCuido - #Shorts,126.044 visualizações,Joel Vieira,Jhonatan Marques,NaN,2021-05-19,7483.0,646.0,126044


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1418 entries, 0 to 1417
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           1418 non-null   object 
 1   views           1418 non-null   object 
 2   cast            1229 non-null   object 
 3   script          136 non-null    object 
 4   directing       92 non-null     object 
 5   date_published  1418 non-null   object 
 6   likes           1410 non-null   float64
 7   dislikes        1410 non-null   float64
 8   views_num       1418 non-null   int32  
dtypes: float64(2), int32(1), object(6)
memory usage: 94.3+ KB


### Substituindo o '\\\n' por vírgula para facilitar dividir atores, roteiristas e diretores em linhas

In [24]:
df['cast'] = df['cast'].replace(r'\\n',',', regex=True)
df['script'] = df['script'].replace(r'\\n',',', regex=True)
df['directing'] = df['directing'].replace(r'\\n',',', regex=True)

In [25]:
df.head()

,title,views,cast,script,directing,date_published,likes,dislikes,views_num
0,EX-FUNCIONÁRIO,404.536 visualizações,"Evelyn Castro,Estevam Nabote,Noemia Oliveira",Pedro Esteves,Zaga Martelletto,2021-05-24,46470.0,379.0,404536
1,VACINADA,527.924 visualizações,Thati Lopes,Gregório Duvivier,Bianca Frossard,2021-05-22,64951.0,1596.0,527924
2,PLANTANANÃ - TUDO SOBRE A CPI DA COVID,322.293 visualizações,Rafael Infante,João Vicente de Castro,Vini Videla,2021-05-21,44357.0,1522.0,322293
3,PFIZER,888.119 visualizações,"Antonio Tabet,Fábio Porchat,João Vicente de Ca...",Gregório Duvivier,Rodrigo Van Der Put,2021-05-20,126596.0,3800.0,888119
4,PREOCUPADÃO COM A COVID #EuMeCuido - #Shorts,126.044 visualizações,Joel Vieira,Jhonatan Marques,NaN,2021-05-19,7483.0,646.0,126044


### Dividindo atores, roteiristas e diretores em linhas

In [27]:
df_exploded = df.assign(cast=df.cast.str.split(",")).explode('cast')
df_exploded2 = df_exploded.assign(script=df_exploded.script.str.split(",")).explode('script')
df_exploded3 = df_exploded2.assign(directing=df_exploded2.directing.str.split(",")).explode('directing')
df_exploded3.shape
df_exploded3.tail(10)

(4366, 9)

,title,views,cast,script,directing,date_published,likes,dislikes,views_num
1414,CSI NOVA IGUAÇU #8,1.183.997 visualizações,Antonio Tabet,Antonio Tabet,Ian SBF,2012-08-07,NaN,NaN,1183997
1414,CSI NOVA IGUAÇU #8,1.183.997 visualizações,Gabriel Totoro,Antonio Tabet,Ian SBF,2012-08-07,NaN,NaN,1183997
1414,CSI NOVA IGUAÇU #8,1.183.997 visualizações,Gustavo Chagas,Antonio Tabet,Ian SBF,2012-08-07,NaN,NaN,1183997
1414,CSI NOVA IGUAÇU #8,1.183.997 visualizações,Letícia Lima,Antonio Tabet,Ian SBF,2012-08-07,NaN,NaN,1183997
1415,COCAÍNA - Porta Dos Fundos Nº 1,2.325.550 visualizações,Julia Rabello,NaN,NaN,2012-08-07,NaN,NaN,2325550
1415,COCAÍNA - Porta Dos Fundos Nº 1,2.325.550 visualizações,Antonio Tabet,NaN,NaN,2012-08-07,NaN,NaN,2325550
1416,TRAVECO DA FIRMA - Porta dos Fundos Nº 1,12.198.167 visualizações,Fabio Porchat,NaN,NaN,2012-08-07,NaN,NaN,12198167
1416,TRAVECO DA FIRMA - Porta dos Fundos Nº 1,12.198.167 visualizações,Gustavo Chagas,NaN,NaN,2012-08-07,NaN,NaN,12198167
1416,TRAVECO DA FIRMA - Porta dos Fundos Nº 1,12.198.167 visualizações,Rafael Infante,NaN,NaN,2012-08-07,NaN,NaN,12198167
1417,PORTA DOS FUNDOS N°1,8.045.907 visualizações,NaN,NaN,NaN,2012-08-06,NaN,NaN,8045907


In [28]:
df_exploded4 = df_exploded3.assign(cast=df_exploded3.cast.str.split("-")).explode('cast')
df_exploded4.shape
df_exploded4.tail(50)

(4389, 9)

,title,views,cast,script,directing,date_published,likes,dislikes,views_num
1408,KKKKKK,4.993.544 visualizações,Gregorio Duvivier,Fábio Porchat,Ian SBF,2012-08-27,108275.0,2831.0,4993544
1408,KKKKKK,4.993.544 visualizações,Gustavo Chagas,Fábio Porchat,Ian SBF,2012-08-27,108275.0,2831.0,4993544
1409,MEIAS PALAVRAS,10.831.846 visualizações,Gregorio Duvivier,Gregorio Duvivier,Ian SBF,2012-08-23,149046.0,5104.0,10831846
1409,MEIAS PALAVRAS,10.831.846 visualizações,Letícia Lima,Gregorio Duvivier,Ian SBF,2012-08-23,149046.0,5104.0,10831846
1409,MEIAS PALAVRAS,10.831.846 visualizações,Luis Lobianco,Gregorio Duvivier,Ian SBF,2012-08-23,149046.0,5104.0,10831846
1409,MEIAS PALAVRAS,10.831.846 visualizações,Rafael Infante,Gregorio Duvivier,Ian SBF,2012-08-23,149046.0,5104.0,10831846
1410,SUPERÁVIT,6.447.232 visualizações,Fábio Porchat,Gregorio Duvivier,Ian SBF,2012-08-20,NaN,NaN,6447232
1410,SUPERÁVIT,6.447.232 visualizações,Marcos Veras,Gregorio Duvivier,Ian SBF,2012-08-20,NaN,NaN,6447232
1411,MODELO VIVO,4.619.487 visualizações,Antonio Tabet,Antonio Tabet,Ian SBF,2012-08-16,NaN,NaN,4619487
1411,MODELO VIVO,4.619.487 visualizações,Gregorio Duvivier,Antonio Tabet,Ian SBF,2012-08-16,NaN,NaN,4619487


### Convertendo o campo "date_published" de string para datetime

In [29]:
df_exploded4['date_published'] = pd.to_datetime(df_exploded4['date_published'].str.slice(start=0, stop=10), format='%Y-%m-%d')
df_exploded4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4389 entries, 0 to 1417
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   title           4389 non-null   object        
 1   views           4389 non-null   object        
 2   cast            4167 non-null   object        
 3   script          542 non-null    object        
 4   directing       330 non-null    object        
 5   date_published  4389 non-null   datetime64[ns]
 6   likes           4368 non-null   float64       
 7   dislikes        4368 non-null   float64       
 8   views_num       4389 non-null   int32         
dtypes: datetime64[ns](1), float64(2), int32(1), object(5)
memory usage: 325.7+ KB


### Convertendo todas as observações das variáveis string para minúsculo
Isso é necessário para evitar que o python trate "Fábio de Luca" como sendo diferente de "Fábio de luca"

In [30]:
df_exploded5 = df_exploded4.apply(lambda x: x.str.lower() if(x.dtype == 'object') else x)
df_exploded5.head()

,title,views,cast,script,directing,date_published,likes,dislikes,views_num
0,ex-funcionário,404.536 visualizações,evelyn castro,pedro esteves,zaga martelletto,2021-05-24,46470.0,379.0,404536
0,ex-funcionário,404.536 visualizações,estevam nabote,pedro esteves,zaga martelletto,2021-05-24,46470.0,379.0,404536
0,ex-funcionário,404.536 visualizações,noemia oliveira,pedro esteves,zaga martelletto,2021-05-24,46470.0,379.0,404536
1,vacinada,527.924 visualizações,thati lopes,gregório duvivier,bianca frossard,2021-05-22,64951.0,1596.0,527924
2,plantananã - tudo sobre a cpi da covid,322.293 visualizações,rafael infante,joão vicente de castro,vini videla,2021-05-21,44357.0,1522.0,322293


### Dropando o campo "views"

In [31]:
df_final = df_exploded5.drop(['views'], axis=1)

In [32]:
df_final.shape
df_final.head()

(4389, 8)

,title,cast,script,directing,date_published,likes,dislikes,views_num
0,ex-funcionário,evelyn castro,pedro esteves,zaga martelletto,2021-05-24,46470.0,379.0,404536
0,ex-funcionário,estevam nabote,pedro esteves,zaga martelletto,2021-05-24,46470.0,379.0,404536
0,ex-funcionário,noemia oliveira,pedro esteves,zaga martelletto,2021-05-24,46470.0,379.0,404536
1,vacinada,thati lopes,gregório duvivier,bianca frossard,2021-05-22,64951.0,1596.0,527924
2,plantananã - tudo sobre a cpi da covid,rafael infante,joão vicente de castro,vini videla,2021-05-21,44357.0,1522.0,322293


### Pronto !!!
O Dataframe acima está pronto para ser analisado. Acrescentei as duas linhas abaixo, caso seja necessário salvar e ler o arquivo em csv.

In [33]:
df_final.to_csv('porta_dos_fundos_longway_cleaned.csv', index=False)

In [ ]:
df_analise = pd.read_csv('porta_dos_fundos_longway_cleaned.csv', parse_dates=['date_published'])

### Conclusão
Este exemplo apresentou uma forma de baixar dados de videos do YouTube sem a necessidade de cadastro no mesmo ou de gerar chave de API. Embora possa ser útil, não é forma mais recomendada pois o código fonte das páginas do YouTube ser alterado a qualquer momento e este método precisará ser reajustado.<br>
Assim, para um uso mais confiável, sugiro que você use a API do YouTube para extrair os dados. Esta forma de extração será apresentada na parte 2 deste tutorial.